In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/79/e1/dcba5ba74392015ceeababf3455138f5875202e66e3316d7ca223bdb7b1c/transformers-4.41.1-py3-none-any.whl.metadata
  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.23.0 from https://files.pythonhosted.org/packages/92/27/1a30d8082ef3c8615ae198b9d451fafffdab815b96727ec3c06befc27ebe/huggingface_hub-0.23.1-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.23.1-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for tokenizers<0.20,>=0.19 from https://files.pythonhosted.org/packages/65/8e/6d7d72b28f22c422cff8beae10ac3c2e4376b9be721ef8167b7eecd1da62/tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/ba/a3/16e9fe3218

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\pablo\\anaconda3\\Lib\\site-packages\\~okenizers\\tokenizers.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [6]:
pip install transformers


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/79/e1/dcba5ba74392015ceeababf3455138f5875202e66e3316d7ca223bdb7b1c/transformers-4.41.1-py3-none-any.whl.metadata
  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)


In [4]:
import torch

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", force_download=True)
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", force_download=True)

AttributeError: module 'pyarrow' has no attribute '__version__'

In [ ]:
df = pd.read_csv('Comment_cleaned_age_gender.csv')

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['comentario'][1])

In [ ]:
def calcular_puntaje_sentimiento(df):
    analysis = []
    for comentario in df['comentario']:
        if isinstance(comentario, str):
            # Truncar el comentario si es demasiado largo
            comentario_truncado = comentario[:512]  # Aquí se asume que el tamaño máximo es 512 tokens
            tokens = tokenizer.encode(comentario_truncado, return_tensors='pt', max_length=512, truncation=True)
            # Evaluar el modelo con el texto truncado
            result = model(tokens)
            # Obtener la salida deseada del modelo
            score = torch.argmax(result.logits).item() + 1
            analysis.append(score)
        else:
            analysis.append(None)  # Otra opción es asignar algún valor para comentarios que no sean cadenas de texto
    df['analysis'] = analysis

In [ ]:
calcular_puntaje_sentimiento(df)

In [ ]:
df['diff'] = df['rating'] - df['analysis']

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df['diff'].value_counts()

In [ ]:
aparts = df[['apart_id', 'comentario']]

In [ ]:
pip install nltk

In [ ]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Descargar los recursos necesarios de NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Definir una función para limpiar los comentarios
def limpiar_comentario(comentario):
    # Eliminar signos de puntuación y las palabras 'estancia' y 'días'
    comentario_sin_puntuacion = re.sub('['+string.punctuation+']', '', comentario)
    comentario_sin_estancia = re.sub(r'\bestancia\b', '', comentario_sin_puntuacion, flags=re.IGNORECASE)
    comentario_sin_dias = re.sub(r'\bdías\b', '', comentario_sin_estancia, flags=re.IGNORECASE)
    return comentario_sin_dias

# Aplicar la función para limpiar los comentarios
aparts['comentario_limpio'] = aparts['comentario'].apply(limpiar_comentario)

# Obtener las stopwords en español
stopwords_espanol = set(stopwords.words('spanish'))

# Paso 1: Agrupa los comentarios por apartamento
comentarios_por_apartamento = aparts.groupby('apart_id')['comentario_limpio'].apply(lambda x: ' '.join(x)).reset_index()

# Paso 2: Combina todos los comentarios correspondientes a cada apartamento en un solo texto
textos_por_apartamento = comentarios_por_apartamento['comentario_limpio'].tolist()

# Paso 3: Tokeniza el texto para dividirlo en palabras y filtra las stopwords
palabras_por_apartamento = [[palabra.lower() for palabra in word_tokenize(texto) if palabra.lower() not in stopwords_espanol] for texto in textos_por_apartamento]

# Paso 4: Realiza un recuento de las palabras para encontrar las más comunes
palabras_comunes_por_apartamento = [Counter(palabras) for palabras in palabras_por_apartamento]

palabras_x_apartamento = pd.DataFrame({'apart_id': comentarios_por_apartamento['apart_id'],
    'palabras_comunes': palabras_comunes_por_apartamento})

In [ ]:
from collections import Counter

# Palabras clave a buscar en los comentarios
palabras_clave = ['en pareja', 'con pareja', 'en familia', 'con hijos', 'con amigos', 'solo']

# Paso 1: Agrupa los comentarios por apartamento
comentarios_por_apartamento = aparts.groupby('apart_id')['comentario_limpio'].apply(lambda x: ' '.join(x)).reset_index()

# Paso 2: Combina todos los comentarios correspondientes a cada apartamento en un solo texto
textos_por_apartamento = comentarios_por_apartamento['comentario_limpio'].tolist()

# Paso 3: Busca la palabra más común entre las palabras clave en los comentarios de cada apartamento
palabra_mas_comun_por_apartamento = []
for texto in textos_por_apartamento:
    contador_palabras_clave = Counter()
    for palabra_clave in palabras_clave:
        contador_palabras_clave[palabra_clave] = texto.count(palabra_clave)
    palabra_mas_comun = contador_palabras_clave.most_common(1)
    palabra_mas_comun_por_apartamento.append(palabra_mas_comun[0][0] if palabra_mas_comun else None)

# Crear un DataFrame con los resultados
df_palabra_mas_comun = pd.DataFrame(palabra_mas_comun_por_apartamento, columns=['Palabra Más Común'])

# Agregar el id del apartamento como índice
df_palabra_mas_comun['apart_id'] = comentarios_por_apartamento['apart_id']
df_palabra_mas_comun = df_palabra_mas_comun.set_index('apart_id')

In [ ]:
print(df_palabra_mas_comun)

In [ ]:
df_palabra_mas_comun['Palabra Más Común'].value_counts()

In [ ]:
df_palabra_mas_comun.to_csv('CompaniaApart.csv')
palabras_x_apartamento.to_csv('PalabrasApart.csv')